In [1]:
%pwd

'c:\\Users\\ENGRACED\\Documents\\Project\\datascience\\farm_copilot\\training\\farm_copilot_model_trainer\\research'

In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\ENGRACED\\Documents\\Project\\datascience\\farm_copilot\\training\\farm_copilot_model_trainer'

In [5]:
from dotenv import load_dotenv

In [6]:
load_dotenv()

True

In [7]:
import tensorflow as tf
from tensorflow import keras

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    path_to_eval_score: Path
    params_image_size: list
    params_batch_size: int

In [9]:
from farm_copilot.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from farm_copilot.utils.common import read_yaml, create_directories


In [11]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        evaluation = self.config.evaluation
        training_data = Path(self.config.data_ingestion.local_data_file)

        create_directories([Path(evaluation.root_dir)])

        eval_config = EvaluationConfig(
            path_of_model=Path(evaluation.path_of_model),
            training_data=training_data,
            mlflow_uri=evaluation.mlflow_uri,
            path_to_eval_score=Path(evaluation.path_to_eval),
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [13]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from src.farm_copilot.utils.common import save_json

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig) -> None:
        self.config = config

    
    @staticmethod
    def load_model(path: Path) -> keras.Model:
        return keras.models.load_model(filepath=path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.load_dataset()
        self.train_validation_test_split()
        self.score = self.model.evaluate(self.test_ds)
        self.save_score()
    

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path(self.config.path_to_eval_score), data=scores)

    
    def load_dataset(self):
        self.dataset = keras.preprocessing.image_dataset_from_directory(
                        directory=self.config.training_data,
                        shuffle=True,
                        label_mode='categorical',
                        image_size = tuple(self.config.params_image_size), 
                        batch_size = self.config.params_batch_size,
                        )
    
    
    def train_validation_test_split(self, shuffle: bool = True, seed: int = 49, shuffle_size: int = 1000):
        # Get the len OF the ds
        ds_size = len(self.dataset)

        # Shuffle datset
        if shuffle:
            self.dataset = self.dataset.shuffle(shuffle_size, seed=49)

        # Compute the ds split sizes
        train_size = int(self.config.params_train_split * ds_size)
        val_size = int(self.config.params_validation_split * ds_size)
        test_size = ds_size - (train_size + val_size)

        # Get the dataset partitions
        self.train_ds = self.dataset.take(train_size)
        self.val_ds = self.dataset.skip(train_size).take(val_size)
        self.test_ds = self.dataset.skip(train_size).skip(val_size).take(test_size)


    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
               mlflow.keras.log_model(self.model, "model", registered_model_name="farm_copilot_xception_model")
            else:
                mlflow.keras.log_model(self.model, "model")
